In [1]:
fastq = "/home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698503_2.fastq.gz"
test_fastq = "/home/laura/projects/virus-watch-data/benchmarking/ebola_benchmark/raw_short/SRR12698503_2.fastq"

outfolder = "/home/laura/projects/virus-watch-data/benchmarking/ebola_benchmark"

kallisto = "/home/laura/projects/kallisto/build/src/kallisto"
bustools = "/home/laura/projects/bustools/build/src/bustools"

### Align test file with kb virus workflow

In [2]:
virus_fasta = "/home/laura/projects/virus-watch-data/virus_ref/uniques_noduplicates.fa"
virus_t2g = "/home/laura/projects/virus-watch-data/virus_ref/uniques_noduplicates_t2g.txt"
virus_index = "/home/laura/projects/virus-watch-data/virus_ref/kallisto_aa/noduplicates_rhesus.idx"
host_fasta = "/home/laura/projects/virus-watch-data/rhesus_ref/Macaca_mulatta.Mmul_10.dna.toplevel.fa.gz"

In [3]:
# # Generate virus reference index
# !/usr/bin/time -v kallisto index \
#     -t 60 \
#     --aa \
#     --d-list=$host_fasta \
#     -i $virus_index \
#     $virus_fasta

In [4]:
# !/usr/bin/time -v kb count \
#     --overwrite \
#     --aa \
#     -t 32 \
#     -i $virus_index \
#     -g $virus_t2g \
#     --parity single \
#     -x default \
#     --h5ad \
#     --kallisto "/home/laura/bin/kallisto" \
#     --bustools "/home/laura/anaconda3/bin/bustools" \
#     -o test \
#     $test_fastq

Same but using kallisto workflow (obtained identical results to kb):

In [53]:
!/usr/bin/time -v $kallisto bus \
        -i $virus_index \
        -o $outfolder/kallisto \
        --aa \
        -n \
        -x bulk \
        -t 32 \
        $test_fastq

!$bustools sort --flags -o $outfolder/kallisto/output_sorted.bus $outfolder/kallisto/output.bus

!$bustools count \
    --genecounts \
    --cm -m \
    -o $outfolder/kallisto/bustools_count/ \
    -g $virus_t2g \
    -e $outfolder/kallisto/matrix.ec \
    -t $outfolder/kallisto/transcripts.txt \
    $outfolder/kallisto/output_sorted.bus


[index] k-mer length: 31
[index] number of targets: 296,623
[index] number of k-mers: 37,541,842
[index] number of distinguishing flanking k-mers: 138
[quant] running in single-end mode
[quant] will process file 1: /home/laura/projects/virus-watch-data/benchmarking/ebola_benchmark/raw_short/SRR12698503_2.fastq
[progress] 99M reads processed (0.4% mapped)             0.4% mapped)              done
[quant] processed 100,000,000 reads, 407,865 reads pseudoaligned

	Command being timed: "/home/laura/projects/kallisto/build/src/kallisto bus -i /home/laura/projects/virus-watch-data/virus_ref/kallisto_aa/uniques_rhesus.idx -o /home/laura/projects/virus-watch-data/benchmarking/ebola_benchmark/kallisto --aa -n -x bulk -t 32 /home/laura/projects/virus-watch-data/benchmarking/ebola_benchmark/raw_short/SRR12698503_2.fastq"
	User time (seconds): 10772.06
	System time (seconds): 18.15
	Percent of CPU this job got: 3087%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 5:49.43
	Average shared text size

Convert bus file to txt file to get indeces of aligned reads:

In [14]:
# To use this, add --flags to bustools sort
!bustools text \
    -f \
    -o $outfolder/kallisto/output_sorted.bus.txt \
    $outfolder/kallisto/output_sorted.bus

Read in 407865 BUS records


### Get number of sequences that aligned to EBOV 

In [61]:
u_tax_tsv = "/home/laura/projects/virus-watch-data/virus_ref/u_tax.tsv"

In [62]:
import anndata
import pandas as pd

In [63]:
# adata = anndata.read(f"{outfolder}/kb/counts_unfiltered/adata.h5ad")
# adata

In [64]:
import kb_python.utils as kb_utils
# Filepath to counts
X = f"{outfolder}/kallisto/bustools_count/output.mtx"
# Filepath to barcode metadata
var_path = f"{outfolder}/kallisto/bustools_count/output.genes.txt"
# Filepath to gene metadata
obs_path = f"{outfolder}/kallisto/bustools_count/output.barcodes.txt" 

# Create AnnData object
adata = kb_utils.import_matrix_as_anndata(X, obs_path, var_path)
adata

/home/laura/anaconda3/lib/python3.9/site-packages/kb_python/utils.py:670: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  anndata.AnnData(X=mtx.tocsr(), obs=df_barcodes, var=df_genes)


AnnData object with n_obs × n_vars = 1 × 296561

In [65]:
adata.X.sum()

76061.0

In [66]:
phylogeny_data = pd.read_csv(u_tax_tsv, sep='\t')
phylogeny_data.head()

,Label,phylum,class,order,family,genus,species
0,u9,Kitrinoviricota,Flasuviricetes,Amarillovirales,Flaviviridae,Flavivirus,West Nile virus
1,u10,Negarnaviricota,Monjiviricetes,Mononegavirales,Filoviridae,Ebolavirus,Zaire ebolavirus
2,u6,Negarnaviricota,Insthoviricetes,Articulavirales,Orthomyxoviridae,Betainfluenzavirus,Influenza B virus
3,u13,Negarnaviricota,Monjiviricetes,Mononegavirales,Rhabdoviridae,Lyssavirus,Rabies lyssavirus
4,u1,Pisuviricota,Pisoniviricetes,Nidovirales,Coronaviridae,Betacoronavirus,Severe acute respiratory syndrome-related coro...


In [71]:
target_ids = phylogeny_data[phylogeny_data["genus"] == "Ebolavirus"]["Label"].values
target_ids

array(['u10', 'u52', 'u361', 'u706', 'u754', 'u1124', 'u1435', 'u2373',
       'u2741', 'u4405', 'u4633', 'u9148', 'u9165', 'u11800', 'u12185',
       'u13484', 'u14834', 'u18263', 'u19179', 'u19221', 'u19688',
       'u20235', 'u25637', 'u26604', 'u27436', 'u27444', 'u28078',
       'u28113', 'u28114', 'u28651', 'u28685', 'u28918', 'u29044',
       'u29578', 'u29871', 'u30841', 'u31076', 'u31079', 'u31093',
       'u31267', 'u34746', 'u35346', 'u36392', 'u36511', 'u36783',
       'u40165', 'u40265', 'u40320', 'u42220', 'u42219', 'u42958',
       'u42986', 'u43271', 'u43270', 'u43728', 'u43774', 'u43779',
       'u43781', 'u55137', 'u55996', 'u103032', 'u144576', 'u150827',
       'u152868', 'u167703', 'u177925', 'u178029', 'u184033', 'u189557',
       'u189954', 'u190559', 'u217252', 'u220027', 'u243208', 'u250613',
       'u276572', 'u276696', 'u280731', 'u292154'], dtype=object)

In [1]:
for id_ in target_ids:
    count = adata[:, adata.var.index == id_].X.sum()
    if count > 0:
        print(f"{id_}: {count}")

NameError: name 'target_ids' is not defined

### Extract sequences that aligned to EBOV 

In [ ]:
# Also see: https://www.biostars.org/p/199946/
from Bio import SeqIO
import pandas as pd
from tqdm import tqdm
TQDM_BAR_FORMAT = (
    "Parsing fastq for reads: {n_fmt} reads processed."
)

def extract_kb_reads(kallisto_out, fastq, gene_id):
    """
    Create new fasta file including only reads that aligned to a specific 
    gene/transcript ID after running kallisto.
    
    Args:
    - kallisto_out    Folder containing kallisto output.
    - fastq           Path to original fastq file.
    - gene_id         ID of gene/transcript of interest.
    
    Returns fasta file.
    
    Note: 
    Only works if `kallisto bus` was run with argument `-n`, 
    `--flags` was included when running `bustools sort`,
    and .bus output was converted to .txt using `bustools text`.
    """
    
    # Get 0-indexed line number of target ID from transcripts.txt
    with open(f"{kallisto_out}/transcripts.txt") as f:
        transcripts = f.read().splitlines()
    gene_id_idx = transcripts.index(gene_id)
    
    # Get equivalence class that matches to 0-indexed line number of target ID
    ec_df = pd.read_csv(f"{kallisto_out}/matrix.ec", sep="\t", header=None)
    gene_id_ec = int(ec_df[ec_df[1] == str(gene_id_idx)][0].values[0])
    
    # Get bus output (converted to txt)
    bus_df = pd.read_csv(f"{kallisto_out}/output_sorted.bus.txt", sep="\t", header=None)
    # Only keep reads that aligned to target ID
    bus_df_target = bus_df[bus_df[2] == gene_id_ec]
    
    # Get numbers of reads that aligned (adjust zero-indexed bus file to one-indexed fastqs)
    reads_aln = bus_df_target[4].values + 1
    
    keep_list = reads_aln.astype(str)

    # Loop over original fastq and write reads with index in keep_list to new fasta
    output_fasta = fastq.split(".fastq")[0] + f"_{gene_id}.fa"

    n_written = 0
    with open(fastq) as handle, open(output_fasta, "w") as out:
        records = SeqIO.parse(handle, "fastq")
        
        with tqdm(bar_format=TQDM_BAR_FORMAT) as pbar:
            for record in records:
                if record.id.split(".")[-1] in keep_list:
                    out.write(">" + record.id + "\n" + str(record.seq) + "\n")
                    n_written += 1
                pbar.update(n=1)
                
    print(f"{n_written} reads saved that matched ID {gene_id}.")

In [ ]:
%%time
extract_kb_reads(f"{outfolder}/kallisto", test_fastq, "u55137")

### Align with bowtie2

In [ ]:
b_index = "/home/laura/projects/kraken2-2.1.2/b_index"

In [ ]:
!/usr/bin/time -v \
    bowtie2 \
    -x $b_index/ebov \
    -f -p 8 \
    -U $outfolder/raw_short/SRR12698503_2_u55137.fa \
    -S $outfolder/kallisto/SRR12698503_2_u55137_EBOV_aligned.sam

### Use SAMtools to convert the SAM files to sorted BAM files

In [ ]:
!/usr/bin/time -v \
    samtools view \
    -bS -F4 $outfolder/kallisto/SRR12698503_2_u55137_EBOV_aligned.sam \
    > $outfolder/kallisto/SRR12698503_2_u55137_EBOV_aligned.bam

In [ ]:
!/usr/bin/time -v \
    samtools sort \
    $outfolder/kallisto/SRR12698503_2_u55137_EBOV_aligned.bam \
    -o $outfolder/kallisto/SRR12698503_2_u55137_EBOV_sorted.bam

In [ ]:
!/usr/bin/time -v \
    samtools index \
    $outfolder/kallisto/SRR12698503_2_u55137_EBOV_sorted.bam